### objective
This is the main notebook of the pipeline that will allow detection and prediction of the rootlet from planarian multiciliated cells

Note 13 April 2021: Data Vizualization is not implemented but I think that data are saved as a csv

## Module Loading

In [2]:
# Allow importation of the others notebook
import import_ipynb

# pyTorch: module for the neural network
import torch
import torch.nn as nn

# load excel files 
# note 13 April 2021: the module might be deprecated
import xlrd  

# time tools
from time import time, asctime

# module that list all files in a input directory
from glob import glob
 
# module that open/save data as csv
import csv

#import asyncio   # note 13/04/2021, this module was already anotated: might be useless

# Data Vizualisation module
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

# helper function 
# note 13/04/2021 instead of * each function should be load independently
from tools.Centriole_Characteristic import *
from tools.Extract_Experiment_Characteristic import *
from tools.CNN_Tools import *
from tools.Graphical_Tools import *
from tools.Centriole_Detection import *

#from tools.Worm_Segmentation import extract_worm_edge

importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Centriole_Characteristic.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/ToolBox.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Midline_Edge_Reformater.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Extract_Experiment_Characteristic.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/CNN_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Graphical_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Analysis_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Centriole_Detection.ipynb


## 63/100x anti-rootletin image(s) and excel file(s)

In [8]:
# This cell load the preprocess 100x images and the corresponding excel file 
# Immunofluorescence images acquired at objective 63x or 100x 
# Excel file with midline and edge coordinate, X- and Y- shift

# 2 options are available: 
optionChosen = 2

# OPTION 1:
# analyzis of all data in a folder
if optionChosen == 1:
    tif_list = glob('./to_analyse/*.tif')
    xls_list = glob('./to_analyse/*.xlsm')

# OPTION 2:
# analyse only a specific file, put the path, and uncomment the two lines below:
elif optionChosen == 2:
    tif_list = ['./full_worm_2/160607_WT_Rootletin_6.xlsm']
    xls_list  = ['./full_worm_2/160607_WT_Rootletin_6.tif']
    
    tif_list = ['./Analyzed/160330_WT_3.xlsm']
    xls_list  = ['./Analyzed/160330_WT_3.tif']
    
else:
    print('Option provided was not understood')
    
    

# WARNING: 
# I'm not sure to put any Quality Check in the script, make sure that the name of both file
# .xlsm and .tif share the same name at the exception of the extension

# Note: 
# It's better to put the full path of the file, the './my_directory' is the linux equivalent of 'c:/my_directory'
# If Option 2: keep the bracket and the apostrophe ['./whateverYouNeed/xxx.xslm']

### Information Cyril the 13/04/2021
 
I annotated the code but i didn't check if it work.
 
From what i looked so far, their is currently no possibility to load some already pre process data (detect centriole one day then predict their angle an other day). This might be implemented

In [9]:
# Do you want to save as image the intermediate result (DOG, find_maxima, DOG+find_maxima)
# It's memory consuming but since this algorithm is very slow it's better to save the result
# especially for publication purpose
SAVE_INTERMEDIATE_IMG = True

# Do you want to load the 'skeleton' of the worm (midline & edge)
LOAD_MIDLINE_AND_EDGE_FROM_EXCEL = True # This should not be ther


# Loop that will analyze one by one all the file define above.
for pathExcel, pathImg_100x in zip(xls_list, tif_list):
    
    # print the time when the file start to be compute and the name of both file
    print(f"{asctime()}")
    print(f"Exl file loaded: {pathExcel}")
    print(f"Tif file loaded: {pathImg_100x}") 

    
# 14/04/2021: The 3 lines below seems to be for future development: automatic edge and midline detection
    # Are you just testing the angle compensation?
    Test_Angle_Compensation = False
    path_img_10x   = './full_worm/C1-180417_Dvl1&2_Odf2-injection_root_17dpa_10x_Full_1.tif'

    
    # Define the method use to predict angle
    problem = 'classification'

    
####################################
# MIDLINE AND WORM EDGE    
####################################

# 13/04/2021: In the future 2 options might be available: 
# Either the midline and edge will be draw manually with ImageJ
# Or an automatic algorithm will do the job
# Currently the automatic version is not available

    # Load Midline and Edge coordinates from a 'classical' excel file
    if LOAD_MIDLINE_AND_EDGE_FROM_EXCEL:

# 13/04/2021: To Raphael: perfect example of why it's better to load function individually and not with a *
# it have been easier for me to find it and know what function is doing exactly

# 13/04/2021: it looks like the get_xls_values() function open the excel file and load the coordinates 
# for midline and edge in this excel file
        db = get_xls_values(pathExcel)

        x_mid = db['worm_midline']['x']
        y_mid = db['worm_midline']['y']
        x_edg = db['worm_edge']['x']
        y_edg = db['worm_edge']['y']

        # Invert the y axis for edge and midline
# 13/04/2021: If i remember well, that's require because imageJ have an invertex y axis (the (0,0) coordinate
# is in top left corner, and we want it bottom left corner)
        newY_mid, newY_edg = [], []
        for y in y_mid:
            newY_mid.append(-y)

        for y in y_edg:
            newY_edg.append(-y)

        # 'skeleton' of the worm
        worm = [x_mid, newY_mid, x_edg, newY_edg ]


    # Automatic characterization of the midline and the edge
    ## So far (17 November 2020) do not work at all. 
    ## Ideas: size of the image, contrast (rm background per example)
    else:
        midline, edge = extract_worm_edge(path_img_10x, quantile = 0.01)
        worm = [midline[0], midline[1], edge[0], edge[1]]

    
    # Reformat midline and Edge in a given number of segment and subsegment
    midline_final = aggregate_segment_char(x_mid, newY_mid, 
                                           x_edg, newY_edg, 
                                           n_midline_seg = 50, 
                                           n_sub_segment = 25, 
                                           n_edge_seg = 200)


    print(f'{asctime()}: Edge and Midline reformated')

# 13/04/2021: improvment possibility of the previous function: perform an interpolation of the midline
# instead of using segment
# might be possible to do it as well for the edge, but perhaps harder because edge is circular


#####################################
# CENTRIOLES IDENTIFICATION
#####################################

    # load the 100x image using OpenCV
    img = cv2.imread(pathImg_100x, cv2.IMREAD_UNCHANGED)
    print(f'{asctime()}: Image Loaded')

    # Compute differential of Gaussian then Otsu thresholding to try to get a mask of the centriole
    # return a binary image
    dog_img = dog_and_otsu(img)
    
    if SAVE_INTERMEDIATE_IMG:
        img_to_save = Image.fromarray(dog_img)
        newPath = pathImg_100x[:-4] + '_dog_otsu.tif'
        img_to_save.save(newPath)
        
    print(f'{asctime()}: Dog & Otsu computed ')

    
    # detect centriole using find_maxima algorithm 
    # note: find_maxima() is the implementation of the find_maxima module in imageJ
    # return a binary image
    find_maxima_img = find_maxima(img)
    
    if SAVE_INTERMEDIATE_IMG:
        img_to_save = Image.fromarray(find_maxima_img)
        newPath = pathImg_100x[:-4] + '_find_maxima.tif'
        img_to_save.save(newPath)
        
    print(f'{asctime()}: Find Maxima Computed')

    
    # combine the 2 previous approaches to get the final 
    # return a binary image
    combine_img = dog_img*find_maxima_img
    
    if SAVE_INTERMEDIATE_IMG:
        img_to_save = Image.fromarray(combine_img)
        newPath = pathImg_100x[:-4] + '_centriole_detected.tif'
        img_to_save.save(newPath)
    print(f'{asctime()}: Centriole detected')

    
# 13/04/2021: I'm almost sure that all the code above this comment work fine.
# Due to some comment in the code, it looks like the code below will not work   # THIS COMMENT NEED TO BE UPDATED 

###########################################
# ANGLE PREDICITON OF THE CENTRIOLE
###########################################

# The prediction is perfomed thanks to a neural network implemented in Python using the pyTorch package

    # Define where the prediction will be perform (on cpu or on gpu (graphic card))
    # using the gpu require an Nvidia graphic card with cuda installed
#13/04/2021: to train a model it's mandatory to use CUDA, to predict the orientation of 'only' 100k
# i don't know how much time it will take to use a cpu. using google Colab might be mandatory
    if torch.cuda.is_available():                                  
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # Loading the Neural Network (VGG_Schmidtea) and the Weight of the network ()
#13/04/2021: only classification is working, the path below do not exist, the appropriate weigth might be:
# './weight/VGG_schmidtea_weight_classification_noReLu_80percent_classification_n72.pth'
    if problem == 'classification':
        model = VGG_Schmidtea(n_classes = 72).to(device)
        #model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_classification.pth', map_location = device))
        model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_classification_noReLu_80percent_classification_n72.pth', map_location = device))

    else:
        model = VGG_Schmidtea(n_classes = 1).to(device)
        model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_regression.pth', map_location = device))

    # Put the model in evaluation/prediction mode
    model.eval()
    

    # Get centriole coordinates from the combine image
    ypts, xpts = np.where(combine_img == 1)
    
    # instantiate a list to store centriole coordinates and predicted angle
    a_list_of_centriole = []
    
    # Note 
    for x, y in zip(xpts, ypts):

        # Ignore centriole located nearby the image edge
        if y > 16 and x > 16 and y < (img.shape[0] - 16) and x < (img.shape[1] - 16):
            # extract a centriole from the image
            centriole = img[y-16:y+16, x-16:x+16]
            
            # transform it in np array in the appropriate type
            centriole = np.asarray(centriole, dtype = "uint8")
            
            # reshape to fit the neural network shape
            centriole = centriole.reshape(1 , 1, 32, 32)
            
            # transform in torch format
            centriole = torch.from_numpy(centriole).float()
            
            # send to the appropriate device
            centriole = centriole.to(device)

            # predict the angle
            with torch.no_grad():
                output = model(centriole)

            # Get the better prediction as np array
            angle = output.max(1)[1].numpy() # might be classification specific

            #print(centriole_extracted)
# 14/04/2021: I didnt' uncomment nor remove predictor() because i don't know what it is suppose to do 
# or if it's an laternative
            #angle = predictor(model, centriole_extracted, device, problem = 'classification')
            a_list_of_centriole.append(((x, y), angle[0]))
        
# 14/04.2021: WARNING code added to save the list as csv: not tested, due to the structure of the variable
# a_list_of_centriole, i'm almost sure that it will not work: the list contain tuples of tuple
# NEED TO BE UPDATED: centriole could be append to the list with: a_list_of_centriole.append([x, y, angle[0]])
# but this require modification later on in the code
    try:
        # Save the predicted centriole localization and angle in a csv 
        with open('pathImg_100x[:-4]_centrioleCoordinates_and_anglePrediction.csv', 'w') as f:
            write = csv.writer(f)
            write.writerow([x_coordinate, y_coordinate, predicted_angle])
            write.writerows(a_list_of_centriole)
    except: 
        print(f"list of centriole coordinates and predicted angle not save in csv")
        
    
# 14/04/2021: I didn't remove the 'compensated' from the print but except if it's in predictor()
# angle orientation has not been compensated for the moment
    # print a message to say that all centriole of the worm have been predicted
    print(f'{asctime()}: Centrioles angle predicted and compensated')


###############################################
# CENTRIOLE REPOSITIONNING ET REORIENTATION
###############################################

    # Load the shift between the 100x image where the centriole have been detected
    # and the 10x image where edge and midline have been draw
    shiftX = db['image_shift']['x']
    shiftY = db['image_shift']['y']
    
    # instantiate a list to save shifted centriole coordinate and predicted angle
    shifted_centriole_list = []

# 14/04/2021: the code below is not optimize at all. Vectorization using np array is better
    # Shift all centriole 
    # as for midline and edge, the obtained Y value have to be inverted due to ImageJ format
    for a_centriole in a_list_of_centriole:
        xShifted = a_centriole[0][0] + shiftX
        yShifted = a_centriole[0][1] + shiftY
        
        # For classification problem transform the class in angle
        if problem == 'classification':
            shifted_centriole_list.append(((xShifted, -yShifted),a_centriole[1]*5+2.5))
        else:
            shifted_centriole_list.append(((xShifted, -yShifted),a_centriole[1]))

            
    # instantiate a list to save centriole coordinates and compensate centrioles angle
    reoriented_centriole = []
    
# 14/04/2021: the code below is not optimize at all. Vectorization using np array is better
# 14/04/2021: I dind't look at centriole_characterizator(), but the code below reorient the predicted angle
# thanks to midline orientation and transform centriole absolute coordinate in relative coordinate
    for a_centriole in shifted_centriole_list:
        tmp_list = list(centriole_characterizator(a_centriole, midline_final))
        if db['worm_orientation'] == 'gauche' or db['worm_orientation'] == 'left':
            tmp_list[-2] = 1 - tmp_list[-2]
            tmp_list[-1] = math.degrees(math.atan2(-math.sin(math.radians(tmp_list[-1])), -math.cos(math.radians(tmp_list[-1]))))
        tmp_list.insert(1,a_centriole[0][0])
        tmp_list.insert(2,a_centriole[0][1])

        reoriented_centriole.append(tmp_list)

# 14/04/2021: It could be usefull to save the results in a excel file for easier 
# need to check if the csv file can be easily open by excel. 
    # Save the centriole relative coordinate and predicted angle as csv
    newPath = pathImg_100x[:-4] + '_DATA.csv'
    with open(newPath, 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(reoriented_centriole)

    print(f'{asctime()}: Centriole Dataset reformated')

    print("\n")
    
# 14/04/2021: End of reviewing for today. Code below seems to be only for data visualization
##################################################################################################
"""
    #######################################################
    # Graphical representation of the results
    # Code above is not mandatory
    #######################################################
    
    # If you want to see some specific centrioles, add them in the list
    list_of_desired_centriole = [0]

    # If you want to see the location of a specific coordinate (Write None if you don't want to see them )
    X_coordinate = None
    Y_coordinate = None

    save_path = pathImg_100x[:-4] + '_schema.tif'
    Worm_And_Centriole(reoriented_centriole, worm, list_of_desired_centriole, (X_coordinate, Y_coordinate), save = True, path = pathImg_100x[:-4])
    
    # Print The graph (worms segmented in 5 antero-posterior parts) + moving average + cstd
    for i in range(5):
        print_a_antero_posterior_result(reoriented_centriole, i, n_ante_post_segment = 5, a_lat_size = 0.1, a_lat_step = 0.05, save = True, path = pathImg_100x[:-4])
        
    # Overlap the analysed image with the identified and analyzed centriole represented as an arrow indicating the predicted angle
    # The starting point of the arrow is the origin of the detected centriole
    # The Ending point indicate the predicted orientation of the centriole

    save_figure = True

    # Each color correspond to a class of 5°
    # So far the color or 'randomly' attributed for each class

    # Define the length of the arrow
    arrowLen = 5

    # Compute as X/Y coordinates the property of the arrow
    # WARNING: 18/11/2020. FOR AN UNKNOWN REASON, the angle is rotated by 90° -> I need to check why
    DATA = []

    for i in a_list_of_centriole:
        angle = i[1] + 90
        x = i[0][0]
        y = i[0][1]
        new_coord = [x-arrowLen*math.cos(math.radians(angle)), y-arrowLen*math.sin(math.radians(angle)), x+arrowLen*math.cos(math.radians(angle)), y+arrowLen*math.sin(math.radians(angle)), angle]
        DATA.append(new_coord)

    DATA = np.array(DATA)

    cmap = plt.cm.jet
    cNorm  = colors.Normalize(vmin=np.min(DATA[:,4]), vmax=np.max(DATA[:,4]))
    scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)

    plt.figure(figsize=(100,50))

    plt.imshow(img)

    for idx in range(0,len(DATA[:,1])):
        colorVal = scalarMap.to_rgba(DATA[idx,4])
        plt.arrow(DATA[idx,0],  #x1
                  DATA[idx,1],  # y1
                  DATA[idx,2]-DATA[idx,0], # x2 - x1
                  DATA[idx,3]-DATA[idx,1], # y2 - y1
                  color=colorVal)
    if save_figure: 
        savePath = pathImg_100x[:-4] + '_Detected_Angle.tif'
        plt.savefig(savePath)
        
    with open('./file_treated.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(tif_list[i])
    print(f"{asctime()}")
    print("\n")

#plt.show()

"""

Wed Apr 14 22:35:49 2021
Exl file loaded: ./Analyzed/160330_WT_3.tif
Tif file loaded: ./Analyzed/160330_WT_3.xlsm


XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'MM\x00*\x00\x00\x00\x08'

In [4]:
# Overlap the analysed image with the identified and analyzed centriole represented as an arrow 
# indicating the predicted angle
# The starting point of the arrow is the origin of the detected centriole
# The Ending point indicate the predicted orientation of the centriole

save_figure = True
show_figure = True

# Each color correspond to a class of 5°
# So far the color or 'randomly' attributed for each class

# Define the length of the arrow
arrowLen = 5

# Compute as X/Y coordinates the property of the arrow
# WARNING: 18/11/2020. FOR AN UNKNOWN REASON, the angle is rotated by 90° -> I need to check why
DATA = []

for i in a_list_of_centriole:
    angle = i[1] + 90
    x = i[0][0]
    y = i[0][1]
    new_coord = [x-arrowLen*math.cos(math.radians(angle)), y-arrowLen*math.sin(math.radians(angle)), x+arrowLen*math.cos(math.radians(angle)), y+arrowLen*math.sin(math.radians(angle)), angle]
    DATA.append(new_coord)
    
DATA = np.array(DATA)

cmap = plt.cm.jet
cNorm  = colors.Normalize(vmin=np.min(DATA[:,4]), vmax=np.max(DATA[:,4]))
scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)

plt.figure(figsize=(100,50))

plt.imshow(img)

for idx in range(0,len(DATA[:,1])):
    colorVal = scalarMap.to_rgba(DATA[idx,4])
    plt.arrow(DATA[idx,0],  #x1
              DATA[idx,1],  # y1
              DATA[idx,2]-DATA[idx,0], # x2 - x1
              DATA[idx,3]-DATA[idx,1], # y2 - y1
              color=colorVal)
if save_figure: 
    savePath = pathImg_100x[:-4] + '_Detected_Angle.tif'
    plt.savefig(savePath)

if show_figure:
    plt.show()  

NameError: name 'a_list_of_centriole' is not defined

In [ ]:
# Graphical Representation of the worms and the analyzed centrioles
# If you want to see some specific centrioles, add them in the list
list_of_desired_centriole = [0]

# If you want to see the location of a specific coordinate (Write None if you don't want to see them )
X_coordinate = None
Y_coordinate = None

save_path = pathImg_100x[:-4] + '_schema.tif'
Worm_And_Centriole(reoriented_centriole, worm, list_of_desired_centriole, (X_coordinate, Y_coordinate), save_path, save = True)

In [ ]:
# Graphical representation of the results 
for i in range(5):
    print_a_antero_posterior_result(reoriented_centriole, i, n_ante_post_segment = 5, a_lat_size = 0.1, a_lat_step = 0.05)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++